In [1]:
import pandas as pd
from pyecharts import options as opts
from pyecharts.charts import Bar,Map, Timeline, Line, Pie, Grid
from pyecharts.globals import ThemeType

In [2]:
from pyecharts.datasets import register_url
register_url("https://echarts-maps.github.io/echarts-countries-js/")

**Shooting per state over time**

In [3]:
data = pd.read_csv("data/df_time.csv")

pop = pd.read_csv("data/acs2015_county_data.csv")
pop = pd.DataFrame(pop.groupby("state_full")['TotalPop'].sum())

abbr = pd.read_csv("data/abre_ful.csv",index_col='state')

shooting = pd.crosstab(data['state'], data['year'], data['race'], aggfunc='count')

shooting = shooting.join(abbr,on = 'state').set_index('state_full')

shooting = shooting.fillna(0)

shooting = shooting.join(pop,on = "state_full")

for y in range(2015,2021):
    shooting[y] = round(shooting[y]/shooting['TotalPop']*1000000,2)

del shooting['TotalPop']

max_=shooting.stack().max()
min_=shooting.stack().min()
print(min_,max_)

0.0 9.54


In [4]:
area_name = shooting.index.tolist()
print(area_name)
print(len(area_name))

['Alaska', 'Alabama', 'Arkansas', 'Arizona', 'California', 'Colorado', 'Connecticut', 'District of Columbia', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Iowa', 'Idaho', 'Illinois', 'Indiana', 'Kansas', 'Kentucky', 'Louisiana', 'Massachusetts', 'Maryland', 'Maine', 'Michigan', 'Minnesota', 'Missouri', 'Mississippi', 'Montana', 'North Carolina', 'North Dakota', 'Nebraska', 'New Hampshire', 'New Jeresy', 'New Mexico', 'Nevada', 'New York', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'SouthCarolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Virginia', 'Vermont', 'Washington', 'Wisconsin', 'West Virginia', 'Wyoming']
51


In [4]:
def get_allcharts(year,shooting):
    max_=shooting.stack().max()
    min_=shooting.stack().min()
    shootingOfYear = shooting.sort_values(by = [year],ascending = False)

    
########bar
    bar = (
        Bar(init_opts = opts.InitOpts(width="1980px",height="1080px",theme=ThemeType.DARK))
        .add_xaxis(shootingOfYear.index.tolist())
        .add_yaxis("Shootings per state",
                   shootingOfYear[year].tolist(),
                   label_opts=opts.LabelOpts(is_show=True,
                                             font_size = 11,
                                             position = "right",
                                             formatter="{b} : {c}")
        )
        .reversal_axis()
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Year {}'s shooting rate per state (*1e-6)".format(year),
                                      pos_left = "center",
                                      padding = 20,
                                      title_textstyle_opts=opts.TextStyleOpts(font_weight="bolder",font_size=36)
                                     ),
            yaxis_opts = opts.AxisOpts(axislabel_opts = opts.LabelOpts(is_show = False)),
            xaxis_opts = opts.AxisOpts(axislabel_opts = opts.LabelOpts(is_show = False)),
            legend_opts = opts.LegendOpts(is_show = False),
            visualmap_opts=opts.VisualMapOpts(is_show=True,is_calculable=True,
                                              dimension = 0,
                                              min_=min_,
                                              max_=max_,
                                              range_text=['high','low'],
                                              pos_left="1%",pos_top="1%"
                                             ),
            graphic_opts=[
                opts.GraphicGroup(
                    graphic_item=opts.GraphicItem(
                        bounding="raw",
                        left="95%",
                        bottom="98%",
                        z=100,
                        is_draggable=False
                    ),
                    children=[
                        opts.GraphicText(
                            graphic_item=opts.GraphicItem(
                                 # 设置字形的 大小，位置
                                left="center",
                                top="center",
                                z=100
                            ),
                        ),
                    ]
                )
            ]
        )
    )
########map
    map0 = (
        Map(init_opts = opts.InitOpts(width="1980px",height="1080px",theme=ThemeType.DARK))
        .add("Shooting", 
             [list(z) for z in zip(shootingOfYear.index.tolist(), shootingOfYear[year].tolist())], 
             "美国",#USA
             label_opts=opts.LabelOpts(is_show=True,font_size = 14),
             center=[-112,34],
             zoom = 0.8
            )
        .set_global_opts(
            legend_opts = opts.LegendOpts(is_show = False),
            visualmap_opts=opts.VisualMapOpts(is_show=True,is_calculable=True,
                                               min_=min_,
                                               max_=max_,
                                               range_text=['high','low'],
                                               pos_left="1%",pos_top="1%"
                                             )
        )
    )
    
    #combination
    grid = (
        Grid(init_opts = opts.InitOpts(width="1980px",height="1080px",theme=ThemeType.DARK))
        .add(bar, grid_opts=opts.GridOpts(pos_top="50%",pos_bottom="4%", pos_left= "2%",pos_right="10%"),
             is_control_axis_index=False)
        .add(map0, grid_opts=opts.GridOpts(pos_top = "1%",pos_bottom="70%",pos_left = "40%",pos_right="1%")
            ,is_control_axis_index=False)
    )
    return grid   

In [10]:
timeline = Timeline(init_opts=opts.InitOpts(width="1980px", height="1080px",theme=ThemeType.DARK))
for y in range(2015, 2021):
    timeline.add(get_allcharts(y,shooting), time_point=str(y))
timeline.add_schema(is_auto_play=True, play_interval=1000,is_timeline_show = True,orient="vertical",width="3%",
                   pos_top = "4%",pos_left = "95%",pos_bottom = "2%")
timeline.render("shooting_2015-2020.html")

'D:\\file\\JupyterFile\\Data2020\\final_report\\shooting_2015-2020.html'

**Shooting rate of black per state**

In [7]:
data = pd.read_csv("df_time.csv")
data = data[data['race'] == 1]

pop = pd.read_csv("acs2015_county_data.csv")
pop = pd.DataFrame(pop.groupby("state_full")['TotalPop'].sum())

abbr = pd.read_csv("abre_ful.csv",index_col='state')

shooting = pd.crosstab(data['state'], data['year'], data['race'], aggfunc='count')

shooting = shooting.join(abbr,on = 'state').set_index('state_full')

shooting = shooting.fillna(0)

shooting = shooting.join(pop,on = "state_full")

for y in range(2015,2021):
    shooting[y] = round(shooting[y]/shooting['TotalPop']*1000000,2)

del shooting['TotalPop']

max_=shooting.stack().max()
min_=shooting.stack().min()
print(min_,max_)

0.0 7.72


In [ ]:
def get_allcharts(year,shooting):
    max_=shooting.stack().max()
    min_=shooting.stack().min()
    shootingOfYear = shooting.sort_values(by = [year],ascending = False)

    
########bar
    bar = (
        Bar(init_opts = opts.InitOpts(width="1980px",height="1080px",theme=ThemeType.DARK))
        .add_xaxis(shootingOfYear.index.tolist())
        .add_yaxis("Shootings per state",
                   shootingOfYear[year].tolist(),
                   label_opts=opts.LabelOpts(is_show=True,
                                             font_size = 11,
                                             position = "right",
                                             formatter="{b} : {c}")
        )
        .reversal_axis()
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Year {}'s shooting rate of black per state (*1e-6)".format(year),
                                      pos_left = "center",
                                      padding = 20,
                                      title_textstyle_opts=opts.TextStyleOpts(font_weight="bolder",font_size=36)
                                     ),
            yaxis_opts = opts.AxisOpts(axislabel_opts = opts.LabelOpts(is_show = False)),
            xaxis_opts = opts.AxisOpts(axislabel_opts = opts.LabelOpts(is_show = False)),
            legend_opts = opts.LegendOpts(is_show = False),
            visualmap_opts=opts.VisualMapOpts(is_show=True,is_calculable=True,
                                              dimension = 0,
                                              min_=min_,
                                              max_=max_,
                                              range_text=['high','low'],
                                              pos_left="1%",pos_top="1%"
                                             ),
            graphic_opts=[
                opts.GraphicGroup(
                    graphic_item=opts.GraphicItem(
                        bounding="raw",
                        left="95%",
                        bottom="98%",
                        z=100,
                        is_draggable=False
                    ),
                    children=[
                        opts.GraphicText(
                            graphic_item=opts.GraphicItem(
                                 # 设置字形的 大小，位置
                                left="center",
                                top="center",
                                z=100
                            ),
                        ),
                    ]
                )
            ]
        )
    )
########map
    map0 = (
        Map(init_opts = opts.InitOpts(width="1980px",height="1080px",theme=ThemeType.DARK))
        .add("Shooting", 
             [list(z) for z in zip(shootingOfYear.index.tolist(), shootingOfYear[year].tolist())], 
             "美国",#USA
             label_opts=opts.LabelOpts(is_show=True,font_size = 14),
             center=[-112,34],
             zoom = 0.8
            )
        .set_global_opts(
            legend_opts = opts.LegendOpts(is_show = False),
            visualmap_opts=opts.VisualMapOpts(is_show=True,is_calculable=True,
                                               min_=min_,
                                               max_=max_,
                                               range_text=['high','low'],
                                               pos_left="1%",pos_top="1%"
                                             )
        )
    )
    
    #combination
    grid = (
        Grid(init_opts = opts.InitOpts(width="1980px",height="1080px",theme=ThemeType.DARK))
        .add(bar, grid_opts=opts.GridOpts(pos_top="50%",pos_bottom="4%", pos_left= "2%",pos_right="10%"),
             is_control_axis_index=False)
        .add(map0, grid_opts=opts.GridOpts(pos_top = "1%",pos_bottom="70%",pos_left = "40%",pos_right="1%")
            ,is_control_axis_index=False)
    )
    return grid   

In [8]:
timeline = Timeline(init_opts=opts.InitOpts(width="1980px", height="1080px",theme=ThemeType.DARK))
for y in range(2015, 2021):
    timeline.add(get_allcharts(y,shooting), time_point=str(y))
timeline.add_schema(is_auto_play=True, play_interval=1000,is_timeline_show = True,orient="vertical",width="3%",
                   pos_top = "4%",pos_left = "95%",pos_bottom = "2%")
timeline.render("shooting_black_2015-2020.html")

'D:\\file\\JupyterFile\\Data2020\\final_report\\shooting_black_2015-2020.html'